<a href="https://colab.research.google.com/github/ChuLinh02/AI4LI/blob/main/MentalQLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import librabries
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
# Load model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2" # Can change by "Qwen1.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Load dataset
def load_dataset(path):
    df = pd.read_csv(path)
    return df

In [ ]:
# Compute perplexity
def calculate_perplexity(text):
  inputs = tokenizer(text, return_tensors="pt",
                     truncation=True, max_length=512).to(device)
  with torch.no_grad():
    outputs = model(**inputs, labels=inputs["input_ids"])
    log_likelihood = outputs.loss
  return np.exp(log_likelihood).item()

In [ ]:
# Filter dataset by perplexity
def filter_by_perplexity(df, column):
  df["perplexity"] = df[column].apply(calculate_perplexity)
  Q1, Q3 = np.percentile(df["perplexity"], [25, 75])
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  return df[(df["perplexity"] >= lower_bound) & (df["perplexity"] <= upper_bound)]

In [ ]:
# Select diverse samples using K-Center Greedy
def k_center_greedy(df, column, k=500):
  embeddings = np.array([tokenizer.encode(text, add_special_tokens=False) for text in df[column]])
  distances = euclidean_distances(embeddings)
  selected = [np.random.choice(len(df))]
  while len(selected) < k:
    max_dist = np.max(np.min(distances[selected], axis=0))
    new_center = np.argmax(np.min(distances[selected], axis=0))
    selected.append(new_center)
  return df.iloc[selected]

In [ ]:
# Testing
